<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C7-Github_Command_And_Control/Setting_Up_A_Github_Account.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Github Command and Control__*

Suppose you've compromised a machine. Now you want it to automatically perform tasks and report its findings back to you. In this chapter, we'll create a __trojan framework__ that will appear innocuous on the remote machine, but we'll be able to assign it all sorts of nefarious tasks.

One of the most challenging aspects of creating a solid trojan framework is figuring out how to control, update, and receive data from you implants. Crucially, you'll need a relatively universal way to push code to your remote trojans. For one thing, this flexibility will let you perform different tasks on each system. Also, you may sometimes need your trojans to selectively run code for certain target operating systems but not others.

Although hackers have devised lots of creative command-and-control methods over the years, relying on technologies such as the Internet Relay Chat (__IRC__) protocol and even __Twitter__, we'll try a service actually designed for code. We'll use __Github__ as a way to store configuration information for our implants and as a means to exfiltrate data from victim systems. Also, we'll host any modules the implant needs to execute tasks on Github. In setting this all up, we'll hack Python's native library-import mechanism so that as you create new trojan modules, your implants can automatically retrieve them, and any dependent libraries, directly from your repo.

Leveraging Github for these tasks can be a clever strategy: Your traffic to Github will be encrypted over Secure Sockets Layer (__SSL__), and we, the authors, have seen very few enterprises actively block Github itself. We'll use a private repo so that prying eyes can't see what we're doing. Once you've coded the capabilities into the trojan, you could theoretically convert it to a binary and drop it on a compromised machine so it runs indefinitely. Then you could use the Github repository to tell it what to do and find what it has discovered.

### *__Setting Up a Github Account__*

If you don't have a Github account, head over to https://github.com/, sign up, and create a new repository called __bhptrojan__. Next, install the Python Github API library (https://pypi.org/project/github3.py/) so that you can automate your interaction with the repo:
```
pip install github3.py
```
Now let's create a basic structure for our repo. Enter the following on the command line:
```
$ mkdir bhptrojan
$ cd bhptrojan
$ git init
$ mkdir modules
$ mkdir config
$ mkdir data
$ touch .gitignore
$ git add .
$ git commit -m "Adds repo structure for trojan."
$ git remote add origin https://github.com/<yourusername>/bhptrojan.git
$ git push origin master
```
Here, we've created the initial structure for the repo. The __config__ directory holds unique configuration files for each trojan. As you deploy trojans, you want each one to perform different tasks, so each trojan will check a separate configuration file. The __module__ directory contains any modular code that the trojan should pick up and then execute. We'll implement a special import hack to allow our trojan to import libraries directly from our Github repo.

This remote load capability will also allow you to stash third-party libraries in Github so you don't have to continually recompile your trojan every time you want to add new functionality or dependencies. The __data__ directory is where the trojan will check in any collected data.

You can create a personal access token on the Github site and use it in place of a password when performing Git operations over __HTTPS__ with the API. The token should provide our trojan with both read and write permissions, since it will need to both read its configuration and write its output. Follow the instructions on the Github site (https://docs.github.com/en/github/authenticating-to-github/) to create the token and save the token string in a local file called __mytoken.txt__. Then, add __mytoken.txt__ to the __.gitignore__ file so you don't accidentally push your credentials to the repository.

Now let's create some simple modules and a sample configuration file.